In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN


import warnings
warnings.filterwarnings("ignore")

/Users/chrisjamal/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_excel('data.xlsx')

In [3]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


In [5]:
df.describe()

,Quantity,Price,Customer ID
count,525461.000000,525461.000000,417534.000000
mean,10.337667,4.688834,15360.645478
std,107.424110,146.126914,1680.811316
min,-9600.000000,-53594.360000,12346.000000
25%,1.000000,1.250000,13983.000000
50%,3.000000,2.100000,15311.000000
75%,10.000000,4.210000,16799.000000
max,19152.000000,25111.090000,18287.000000


In [6]:
df.isnull()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
525456,False,False,False,False,False,False,False,False
525457,False,False,False,False,False,False,False,False
525458,False,False,False,False,False,False,False,False
525459,False,False,False,False,False,False,False,False


In [7]:
missing_values = df.isnull().sum()

missing_values

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [8]:
#Exclude rows with missing Customer ID
df = df.dropna(subset=['Customer ID'])

In [9]:
df['Description'] = df['Description'].fillna('Missing Description')

In [10]:
missing_values = df.isnull().sum()

missing_values

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [11]:
# Identify non-numeric columns
non_numeric_columns = df.select_dtypes(exclude=['int', 'float']).columns

# Drop non-numeric columns (if they are not needed for clustering)
df_numeric = df.drop(non_numeric_columns, axis=1)


In [12]:
# Check for non-numeric values in what are supposed to be numeric columns
for col in df_numeric.columns:
    if not pd.to_numeric(df_numeric[col], errors='coerce').notnull().all():
        print(f"Column {col} contains non-numeric values.")
        # Handle the non-numeric values accordingly
        # For example, replace them with NaN and then impute or remove
        df_numeric[col] = pd.to_numeric(df_numeric[col], errors='coerce')


In [16]:
# Step 1: Select only numeric columns (this step might be redundant if 'df' is already all numeric)
df_numeric = df.select_dtypes(include=[np.number])

# Step 2: Scale the data
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_numeric)

# Step 3: Apply K-Means Clustering
kmeans = KMeans(n_clusters=3, random_state=42)
cluster_labels = kmeans.fit_predict(df_scaled)

# Outputting the cluster labels
print(cluster_labels)


[0 0 0 ... 2 2 2]


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417534 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      417534 non-null  object        
 1   StockCode    417534 non-null  object        
 2   Description  417534 non-null  object        
 3   Quantity     417534 non-null  int64         
 4   InvoiceDate  417534 non-null  datetime64[ns]
 5   Price        417534 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      417534 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 28.7+ MB


In [18]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [19]:
# Create a Total Spend feature
df['Total Spend'] = df['Quantity'] * df['Price']

# Create features from InvoiceDate
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['Day'] = df['InvoiceDate'].dt.day
df['Hour'] = df['InvoiceDate'].dt.hour
df['Weekday'] = df['InvoiceDate'].dt.weekday

# Aggregate at the customer level, if needed
customer_features = df.groupby('Customer ID').agg({
    'Total Spend': 'sum',   # Total spend per customer
    'Invoice': 'nunique',   # Count of unique invoices per customer as a proxy for frequency
    'Quantity': 'sum'       # Total quantity of products purchased
    # ... you can add more aggregations as needed
}).reset_index()

In [20]:
for column in df.columns:
    # Attempt to convert each column to numeric, coercing errors to NaN
    temp = pd.to_numeric(df[column], errors='coerce')
    # Check if there are any nulls only after conversion (indicating conversion failures)
    if temp.isnull().sum() > 0:
        print(f"Column '{column}' contains non-numeric values.")


Column 'Invoice' contains non-numeric values.
Column 'StockCode' contains non-numeric values.
Column 'Description' contains non-numeric values.
Column 'Country' contains non-numeric values.


In [ ]:
import pandas as pd

# Assuming 'df' is your DataFrame
# Apply one-hot encoding to the specified categorical columns
df_encoded = pd.get_dummies(df, columns=['Invoice', 'StockCode', 'Description', 'Country'])

# df_encoded now contains the original numerical data and one-hot encoded categorical features


In [ ]:
# Scale the encoded data
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_encoded)

# Apply PCA
pca = PCA(n_components=0.95)  # Retain 95% of the variance
df_reduced = pca.fit_transform(df_scaled)

In [ ]:
# Assuming your df has only the features to cluster
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

# Choose the number of clusters (k) and fit K-Means
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(df_scaled)

# The cluster labels for each data point
cluster_labels = kmeans.labels_

In [ ]:
# Normalize data
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

# Apply hierarchical clustering
hierarchical = AgglomerativeClustering(n_clusters=3)
cluster_labels = hierarchical.fit_predict(df_scaled)

In [ ]:
# Normalize data
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

# Apply DBSCAN
# eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other
# min_samples: The number of samples in a neighborhood for a point to be considered as a core point
dbscan = DBSCAN(eps=0.5, min_samples=5)
cluster_labels = dbscan.fit_predict(df_scaled)